In [22]:
import pandas  as pd
import string
from nltk.corpus import stopwords

In [13]:
#Get the spam data collection 
spam_data= pd.read_csv('SpamCollection',sep='\t', names=['response', 'Message'])

In [14]:
spam_data(

,response,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
#finding more data using describe function
spam_data.describe()


,response,Message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [16]:
#view response using groupby and describe method
spam_data.groupby('response'). describe()

Message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [17]:
#Verify length of the messages and also add it as a new column 
spam_data['Length']= spam_data['Message']. apply(len)

In [19]:
spam_data.head()

,response,Message,Length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [20]:
#define a function to get rid of stopwords present in the messages
def text_process(mess):
    no_pun=[char for char in mess if char not in string.punctuation]
    no_pun=''.join(no_pun)
    return[word for word in no_pun.split() if word.lower() not in stopwords.words('english')]

In [23]:
import nltk
nltk.download('stopwords')
spam_data['Message'].head().apply(text_process)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Naina\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: Message, dtype: object

In [24]:
#start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
#use bag of words by applying the function and fit the data into it
bag_of_words=CountVectorizer(analyzer=text_process).fit(spam_data['Message'])

In [26]:
#print length of bag of words stored in the vocabulary_ attribute
print(len(bag_of_words.vocabulary_))

11425


In [27]:
message_bow= bag_of_words.transform(spam_data['Message'])

In [28]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidftransformer=TfidfTransformer().fit(message_bow)

In [29]:
#print shape of the tfidf 
message_tfidf=tfidftransformer.transform(message_bow)
print(message_tfidf.shape)

(5572, 11425)


In [30]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_model= MultinomialNB().fit(message_tfidf,spam_data['response'])

In [36]:
#check model for the predicted and expected value say for message#2 and message#5
message= spam_data['Message'][3]
bagofwords=bag_of_words.transform([message])
tfidf=tfidftransformer.transform(bagofwords)
print("predicted", spam_model.predict(tfidf)[0])
print("actual", spam_data['response'][3])

predicted ham
actual ham
